In [ ]:
!pip install compressai torch torchvision Pillow tqdm --quiet

# restart

In [8]:
import torch
from compressai.zoo import cheng2020_anchor
from torchvision import transforms
from PIL import Image
import os

# CPU kullan (GPU’da OOM hatası alıyorduk)
device = torch.device("cpu")

qualities = [1, 2, 3, 4, 5]
input_dir = "/kaggle/input/anadolu-deneme/deneme"
base_output_dir = "/kaggle/working/compai_outputs"
os.makedirs(base_output_dir, exist_ok=True)

transform = transforms.Compose([
    transforms.Resize((256, 256)),  # boyutu küçült
    transforms.ToTensor(),
])

for quality in qualities:
    model = cheng2020_anchor(quality=quality, metric='mse', pretrained=True).to(device).eval()
    
    for fname in sorted(os.listdir(input_dir)):
        img = Image.open(os.path.join(input_dir, fname)).convert("RGB")
        x = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            encoded = model.compress(x)
            decoded = model.decompress(encoded["strings"], encoded["shape"])
            x_hat = decoded["x_hat"]

        recon = x_hat.squeeze().permute(1, 2, 0).detach().cpu().numpy()
        recon = (recon * 255).round().astype("uint8")

        # ✅ quality bilgisi eklendi
        out_fname = f"recon_q{quality}_{fname}"
        Image.fromarray(recon).save(os.path.join(base_output_dir, out_fname))

print("✅ Tüm quality seviyeleri için reconstruct işlemi tamamlandı.")


Downloading: "https://compressai.s3.amazonaws.com/models/v1/cheng2020-anchor-2-a29008eb.pth.tar" to /root/.cache/torch/hub/checkpoints/cheng2020-anchor-2-a29008eb.pth.tar
100%|██████████| 49.1M/49.1M [00:03<00:00, 13.5MB/s]
Downloading: "https://compressai.s3.amazonaws.com/models/v1/cheng2020-anchor-3-e49be189.pth.tar" to /root/.cache/torch/hub/checkpoints/cheng2020-anchor-3-e49be189.pth.tar
100%|██████████| 49.1M/49.1M [00:03<00:00, 15.3MB/s]
Downloading: "https://compressai.s3.amazonaws.com/models/v1/cheng2020-anchor-4-98b0b468.pth.tar" to /root/.cache/torch/hub/checkpoints/cheng2020-anchor-4-98b0b468.pth.tar
100%|██████████| 109M/109M [00:06<00:00, 18.5MB/s] 
Downloading: "https://compressai.s3.amazonaws.com/models/v1/cheng2020-anchor-5-23852949.pth.tar" to /root/.cache/torch/hub/checkpoints/cheng2020-anchor-5-23852949.pth.tar
100%|██████████| 109M/109M [00:07<00:00, 14.7MB/s] 


✅ Tüm quality seviyeleri için reconstruct işlemi tamamlandı.


In [9]:
import zipfile
import os

output_dir = "/kaggle/working/compai_outputs"
zip_path = "/kaggle/working/compressai_outputs.zip"

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for fname in os.listdir(output_dir):
        fpath = os.path.join(output_dir, fname)
        zipf.write(fpath, arcname=fname)

print(f"ZIP dosyası oluşturuldu: {zip_path}")


ZIP dosyası oluşturuldu: /kaggle/working/compressai_outputs.zip
